In [44]:
import pandas as pd
import geopandas as geopd

import os
import pathlib

from tqdm.notebook import tqdm

In [42]:
order_list = ['discharge_vol', 'discharge_spec', 'precipitation', 'pet', 'snow_depth', 'swe', 'temperature_gmin', 'temperature_min', 'temperature_mean', 'temperature_max', 'humidity_rel', 'radiation_global']

catchments_path = "/home/iielse/Documents/DIWA/data/CAMELS-FI catchments/CAMELS-FI_catchments.gpkg"
src_root = "/media/iielse/T9/CAMELS-FI/data/timeseries_by_attribute"
dst_root = "/media/iielse/T9/CAMELS-FI/data/timeseries"

catchments = geopd.read_file(catchments_path, layer='v1')

root_path = pathlib.Path(src_root)
# Getting the files in 
files = [item for item in root_path.iterdir() if item.is_file()]

In [37]:
files

[PosixPath('/media/iielse/T9/CAMELS-FI/data/timeseries_by_attribute/precipitation.csv'),
 PosixPath('/media/iielse/T9/CAMELS-FI/data/timeseries_by_attribute/temperature_min.csv'),
 PosixPath('/media/iielse/T9/CAMELS-FI/data/timeseries_by_attribute/swe.csv'),
 PosixPath('/media/iielse/T9/CAMELS-FI/data/timeseries_by_attribute/temperature_mean.csv'),
 PosixPath('/media/iielse/T9/CAMELS-FI/data/timeseries_by_attribute/temperature_max.csv'),
 PosixPath('/media/iielse/T9/CAMELS-FI/data/timeseries_by_attribute/radiation_global.csv'),
 PosixPath('/media/iielse/T9/CAMELS-FI/data/timeseries_by_attribute/snow_depth.csv'),
 PosixPath('/media/iielse/T9/CAMELS-FI/data/timeseries_by_attribute/temperature_gmin.csv'),
 PosixPath('/media/iielse/T9/CAMELS-FI/data/timeseries_by_attribute/pet.csv'),
 PosixPath('/media/iielse/T9/CAMELS-FI/data/timeseries_by_attribute/humidity_rel.csv'),
 PosixPath('/media/iielse/T9/CAMELS-FI/data/timeseries_by_attribute/discharge_spec.csv'),
 PosixPath('/media/iielse/T9/CA

In [46]:
# Some of the files contain too many catchments (to short flow time series), so the correct ones are read from the definitely correct place.
gauges = list(catchments['Paikka_Id'])

for gauge in tqdm(gauges):
    columns = []
    
    for file in files:
        attribute = file.stem
        meteo = pd.read_csv(file)
        meteo['date'] = pd.to_datetime(meteo['date']) 
        meteo = meteo.set_index('date')
        column = meteo[[gauge]]
        column = column.rename({gauge: attribute}, axis=1)
        columns.append(column)
    output = pd.concat(columns, axis=1)
    # reordering the columns
    output = output[order_list]
    dst_path = os.path.join(dst_root, f"CAMELS_FI_hydromet_timeseries_{gauge}_19610101-20231231.csv")
    output.to_csv(dst_path)
    break

  0%|          | 0/320 [00:00<?, ?it/s]

In [45]:
dst_path

'/media/iielse/T9/CAMELS-FI/data/timeseries/CAMELS_FI_hydromet_timeseries_996_19610101-20231231.csv'